# AWS comprenend medical with Python

### Tutorial
https://docs.aws.amazon.com/comprehend/latest/dg/get-started-api-med.html

#### boto3 documentation
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/comprehendmedical.html

In [1]:
import boto3
client = boto3.client(service_name='comprehendmedical', region_name='us-east-1')
result = client.detect_entities(Text= 'cerealx 84 mg daily')
entities = result['Entities'];
for entity in entities:
    print('Entity', entity)

Entity {'Id': 0, 'BeginOffset': 0, 'EndOffset': 7, 'Score': 0.8877691626548767, 'Text': 'cerealx', 'Category': 'MEDICATION', 'Type': 'BRAND_NAME', 'Traits': [], 'Attributes': [{'Type': 'DOSAGE', 'Score': 0.9337134957313538, 'RelationshipScore': 0.9995118379592896, 'Id': 1, 'BeginOffset': 8, 'EndOffset': 13, 'Text': '84 mg', 'Traits': []}, {'Type': 'FREQUENCY', 'Score': 0.990627646446228, 'RelationshipScore': 0.9987651109695435, 'Id': 2, 'BeginOffset': 14, 'EndOffset': 19, 'Text': 'daily', 'Traits': []}]}


In [5]:
result['Entities']

[{'Id': 0,
  'BeginOffset': 0,
  'EndOffset': 7,
  'Score': 0.8877691626548767,
  'Text': 'cerealx',
  'Category': 'MEDICATION',
  'Type': 'BRAND_NAME',
  'Traits': [],
  'Attributes': [{'Type': 'DOSAGE',
    'Score': 0.9337134957313538,
    'RelationshipScore': 0.9995118379592896,
    'Id': 1,
    'BeginOffset': 8,
    'EndOffset': 13,
    'Text': '84 mg',
    'Traits': []},
   {'Type': 'FREQUENCY',
    'Score': 0.990627646446228,
    'RelationshipScore': 0.9987651109695435,
    'Id': 2,
    'BeginOffset': 14,
    'EndOffset': 19,
    'Text': 'daily',
    'Traits': []}]}]

## Example with mtsample.com data

In [6]:
note = "SUBJECTIVE: This 23-year-old white female presents with complaint of allergies. She used to have allergies when she lived in Seattle but she thinks they are worse here. In the past, she has tried Claritin, and Zyrtec. Both worked for short time but then seemed to lose effectiveness. She has used Allegra also. She used that last summer and she began using it again two weeks ago. It does not appear to be working very well. She has used over-the-counter sprays but no prescription nasal sprays. She does have asthma but doest not require daily medication for this and does not think it is flaring up.MEDICATIONS: Her only medication currently is Ortho Tri-Cyclen and the Allegra.ALLERGIES: She has no known medicine allergies.OBJECTIVE:Vitals: Weight was 130 pounds and blood pressure 124/78.HEENT: Her throat was mildly erythematous without exudate. Nasal mucosa was erythematous and swollen. Only clear drainage was seen. TMs were clear.Neck: Supple without adenopathy.Lungs: Clear.ASSESSMENT: Allergic rhinitis.PLAN:1. She will try Zyrtec instead of Allegra again. Another option will be to use loratadine. She does not think she has prescription coverage so that might be cheaper.2. Samples of Nasonex two sprays in each nostril given for three weeks. A prescription was written as well."
mt_note = client.detect_entities(Text = note)

In [8]:
mt_note["Entities"]

[{'Id': 6,
  'BeginOffset': 17,
  'EndOffset': 19,
  'Score': 0.9988079071044922,
  'Text': '23',
  'Category': 'PROTECTED_HEALTH_INFORMATION',
  'Type': 'AGE',
  'Traits': []},
 {'Id': 23,
  'BeginOffset': 69,
  'EndOffset': 78,
  'Score': 0.5537630915641785,
  'Text': 'allergies',
  'Category': 'MEDICAL_CONDITION',
  'Type': 'DX_NAME',
  'Traits': []},
 {'Id': 7,
  'BeginOffset': 125,
  'EndOffset': 132,
  'Score': 0.9995848536491394,
  'Text': 'Seattle',
  'Category': 'PROTECTED_HEALTH_INFORMATION',
  'Type': 'ADDRESS',
  'Traits': []},
 {'Id': 9,
  'BeginOffset': 196,
  'EndOffset': 204,
  'Score': 0.9996539354324341,
  'Text': 'Claritin',
  'Category': 'MEDICATION',
  'Type': 'BRAND_NAME',
  'Traits': []},
 {'Id': 10,
  'BeginOffset': 210,
  'EndOffset': 216,
  'Score': 0.9988848567008972,
  'Text': 'Zyrtec',
  'Category': 'MEDICATION',
  'Type': 'BRAND_NAME',
  'Traits': []},
 {'Id': 24,
  'BeginOffset': 264,
  'EndOffset': 282,
  'Score': 0.4836021065711975,
  'Text': 'lose effe

In [14]:
aaa = mt_note["Entities"]
len(aaa)

34

In [16]:
for a in aaa:
    

list